In [12]:
import torch
import timm
from torchvision import transforms
from PIL import Image
import numpy as np
import os
from matplotlib import pyplot as plt
import shutil

In [2]:


device = "cuda"

model = timm.create_model(
    "vit_base_patch14_dinov2",
    pretrained=True,
    num_classes=0
).to(device).eval()

transform = transforms.Compose([
    transforms.Resize(518),
    transforms.CenterCrop(518),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
    ),
])


In [3]:

def embed_image(path):
    img = Image.open(path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        return model(x).cpu().numpy()[0]


In [6]:
IMAGE_LIST = sorted(os.listdir("ChatExport_2026-01-31/photos/"))

In [7]:
def get_all_embeddings():
    screenshots_dir = "ChatExport_2026-01-31/photos/"
    embeddings = []
    for file_name in IMAGE_LIST:
        file_path = os.path.join(screenshots_dir, file_name)
        embed = embed_image(file_path)
        embeddings.append(embed)
    embeddings = np.array(embeddings)
    return embeddings

embeddings = get_all_embeddings()

In [18]:
K_centrs = np.random.rand(10, 768)

In [20]:
def claster_embedings(df, K=10, K_centrs=None):
    dim = df.shape[1]
    # K_centrs = np.random.rand(K, dim)
    klasters_idx = [[] for _ in range(K)]
    for u in range(10):
        for i, p in enumerate(df):
            argmin = np.argmin([np.linalg.norm(K - p) for K in K_centrs])
            klasters_idx[argmin].append(i)
        for i, K in enumerate(K_centrs):
            if klasters_idx[i]:
                K_centrs[i] = df[klasters_idx[i]].mean(0)
    return K_centrs
K_centrs = claster_embedings(embeddings, 10, K_centrs)

In [21]:
def make_klaster_folder(K, df):
    klster_dir = "clasters"
    if os.path.exists(klster_dir):
        shutil.rmtree(klster_dir)
    os.mkdir(klster_dir)
    photo_dir = 'ChatExport_2026-01-31/photos'
    file_dirs = [os.path.join(photo_dir, file_name) for file_name in IMAGE_LIST]
    for i, p in enumerate(df):
        argmin = str(int(np.argmin([np.linalg.norm(k - p) for k in K])))
        sub_klaster_dir = os.path.join(klster_dir, argmin)
        if not os.path.exists(sub_klaster_dir):
            os.mkdir(sub_klaster_dir)
        os.system(f"cp {file_dirs[i]} {os.path.join(sub_klaster_dir, IMAGE_LIST[i])}")
make_klaster_folder(K_centrs, embeddings)

In [22]:
claster_cats=[
    0,#0
    0,#1
    0,#2
    1,#3
    1,#4
    1,#5
    1,#6
    0,#7
    1,#8
    1,#9
]

In [25]:
with open("K_centrs.npy", 'wb') as f:
    np.save(f, K_centrs)